# Description
This notebook aims to compare the models performance accross different segments

# Setup

In [14]:
import pandas as pd
import plotly.express as px
import re
import numpy as np
from IPython.display import Image
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import joblib
import shap

import findspark
import pyspark
from pyspark.sql import SparkSession
import pyspark
from pyspark.sql import DataFrame
import pyspark.sql.functions as sql
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.functions import vector_to_array

from sklearn.base import BaseEstimator, ClassifierMixin

In [4]:
findspark.init()
sc = pyspark.SparkContext(appName="Test")
spark = SparkSession.builder.master("local[*]") \
                    .appName('test') \
                    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/09 13:26:29 WARN Utils: Your hostname, pop-os-note, resolves to a loopback address: 127.0.0.1; using 192.168.0.4 instead (on interface wlp2s0)
25/06/09 13:26:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/09 13:26:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/09 13:26:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/09 13:26:36 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [5]:
input_path = "../data/"
output_path = "../outputs/"

# Data

In [7]:
train_data = spark.read.parquet(f"{output_path}train_df_woe.parquet")
test_data  = spark.read.parquet(f"{output_path}test_df_woe.parquet") 

# Models

In [15]:
log_reg = PipelineModel.load(f"{output_path}_log_reg.gz")

In [ ]:
class pysparkLogRegWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, pyspark_pipeline: PipelineModel):
        self.pipeline_model = pyspark_pipeline
        self.feature_name_ = self.pipeline_model.stages[0].getInputCols()
        self.feature_name_in_ = self.pipeline_model.stages[0].getInputCols()
        self.classes_ = np.array([0,1], dtype=int)
        self.params = 
        

log_reg = PipelineModel.load(f"{output_path}_log_reg.gz")

# Analysis

In [11]:
selected_cols = \
['funded_amnt_woe',
 'term_woe',
 'sub_grade_woe',
 'home_ownership_woe',
 'annual_inc_woe',
 'verification_status_woe',
 'zip_code_woe',
 'dti_woe',
 'earliest_cr_line_woe',
 'fico_range_low_woe',
 'inq_last_6mths_woe',
 'revol_util_woe']

target_col = "default_flag"

train_pd = train_data.select(
    selected_cols + [target_col]
)\
.toPandas()

In [13]:
explainer = shap.LinearExplainer(log_reg.stages[1], train_pd)

InvalidModelError: An unknown model type was passed: <class 'pyspark.ml.regression.GeneralizedLinearRegressionModel'>

## SHAP

In [ ]:
class applyShap:
    def __init__(self,
                 model,
                 data):
        self.model = model
        self.data = data
    
    def explain(self):
        try:
            self.explainer = shap.TreeExplainer(self.model, feature_perturbation="tree_path_dependent")
        except:
            self.explainer = shap.LinearExplainer(self.mode, self.data, feature_names = self.model.features_names_in_)
        self.shap_values = self.explainer(self.data)
    
    def calculate_shap_values(self):
        self.shap_values = self.explainer(self.data)
    
    def beeswarm_plot(self, max_display=20, log_scale=False):
        try:
            shap.plots.beeswarm(self.shap_values, max_display = max_display, log_scale = log_scale)
        except:
            shap.plots.beeswarm(self.shap_values[:,:,1], max_display = max_display, log_scale = log_scale)